In [1]:
import pandas as pd
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
from cassandra.cluster import Cluster
import subprocess

In [4]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address        Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.144.3  70.27 KiB  16      60.0%             ec7f5b01-8ee7-493e-bea3-ef75c949e4b8  rack1
UN  192.168.144.4  25.55 KiB  16      80.0%             14acab58-d159-4c7f-a753-2a65b482ca27  rack1
UN  192.168.144.2  25.55 KiB  16      60.0%             5c25eae9-4e89-4bf0-a856-db1b75d86ba8  rack1



In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [6]:
cass.execute("DROP KEYSPACE IF EXISTS weather")

In [7]:
cass.execute("create keyspace weather with replication = {'class' : 'SimpleStrategy', 'replication_factor' : 3}")

In [8]:
cass.execute("use weather")

In [9]:
cass.execute("""
CREATE TYPE station_record (tmin INT,tmax INT)
""")

In [10]:
cass.execute("drop table if exists stations")

In [11]:
cass.execute("""
create table stations(
    id TEXT,
    name TEXT static,
    date DATE,
    record weather.station_record,
    PRIMARY KEY (id , date)
)
""")

In [12]:
#q1
cass.execute("describe table stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [13]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a698b80c-fa2f-49c5-bad6-86e7dca0fb4d;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [14]:
df = spark.read.text("ghcnd-stations.txt")

In [15]:
df2 = df.withColumn("id", expr("SUBSTRING(value, 1, 11)")) \
        .withColumn("state", expr("SUBSTRING(value, 39, 2)")) \
        .withColumn("name", expr("SUBSTRING(value, 41, 30)"))

In [16]:
df2.limit(10).toPandas()

,value,id,state,name
0,ACW00011604 17.1167 -61.7833 10.1 ST JO...,ACW00011604,,ST JOHNS COOLIDGE FLD
1,ACW00011647 17.1333 -61.7833 19.2 ST JO...,ACW00011647,,ST JOHNS
2,AE000041196 25.3330 55.5170 34.0 SHARJ...,AE000041196,,SHARJAH INTER. AIRP
3,AEM00041194 25.2550 55.3640 10.4 DUBAI...,AEM00041194,,DUBAI INTL
4,AEM00041217 24.4330 54.6510 26.8 ABU D...,AEM00041217,,ABU DHABI INTL
5,AEM00041218 24.2620 55.6090 264.9 AL AI...,AEM00041218,,AL AIN INTL
6,AF000040930 35.3170 69.0170 3366.0 NORTH...,AF000040930,,NORTH-SALANG
7,AFM00040938 34.2100 62.2280 977.2 HERAT...,AFM00040938,,HERAT
8,AFM00040948 34.5660 69.2120 1791.3 KABUL...,AFM00040948,,KABUL INTL
9,AFM00040990 31.5000 65.8500 1010.0 KANDA...,AFM00040990,,KANDAHAR AIRPORT


In [17]:
filtered_stations_df = df2.filter(col("state") == "WI")
filtered_stations_df.limit(10).toPandas()

,value,id,state,name
0,US1WIAD0002 43.9544 -89.8096 294.4 WI ADAMS...,US1WIAD0002,WI,ADAMS 0.4 E
1,US1WIAD0005 44.2053 -89.8480 305.7 WI NEKOO...,US1WIAD0005,WI,NEKOOSA 8.0 SSE
2,US1WIAD0006 43.8858 -89.7259 307.8 WI GRAND...,US1WIAD0006,WI,GRAND MARSH 1.0 W
3,US1WIAD0008 43.8611 -89.7163 310.0 WI GRAND...,US1WIAD0008,WI,GRAND MARSH 1.9 SSW
4,US1WIAD0010 43.7864 -89.6417 293.8 WI OXFOR...,US1WIAD0010,WI,OXFORD 4.0 W
5,US1WIAD0015 43.7855 -89.6483 288.3 WI OXFOR...,US1WIAD0015,WI,OXFORD 4.3 W
6,US1WIAD0016 44.0623 -89.7948 315.5 WI FRIEN...,US1WIAD0016,WI,FRIENDSHIP 6.4 NNE
7,US1WIAD0017 44.0628 -89.8001 302.7 WI FRIEN...,US1WIAD0017,WI,FRIENDSHIP 6.4 N
8,US1WIAS0003 46.5818 -90.8842 210.0 WI ASHLA...,US1WIAS0003,WI,ASHLAND 0.5 WNW
9,US1WIAS0004 46.5824 -90.8839 213.4 WI ASHLA...,US1WIAS0004,WI,ASHLAND 0.5 WNW


In [18]:
loop_data = filtered_stations_df.collect()

In [19]:
# loop_data

In [20]:
for row in loop_data:
    station_id = row["id"].strip()
    station_name = row["name"].strip()
    station_name = re.sub(r'[^\w]', ' ', station_name)
    query = "INSERT INTO weather.stations (id, name) VALUES ('{}', '{}')".format(station_id, station_name)
    cass.execute(query)

In [21]:
x = cass.execute("""
SELECT count(*) FROM weather.stations
""")

In [22]:
cnt = pd.DataFrame(x)

In [23]:
cnt

,count
0,1313


In [24]:
#q2
x = cass.execute("""
select name from stations where id='USW00014837'
""")
y=x.one()[0]
y

'MADISON DANE CO RGNL AP'

In [25]:
#q3
x = cass.execute("""
select token(id) from stations where id='USC00470273'
""")
y=x.one()[0]
y

-9014250178872933741

In [26]:
!nodetool ring  -r -- weather



Datacenter: datacenter1
Address             Rack        Status State   Load            Owns                Token                                       
                                                                                   8899360145757685173                         
p6-db-2.project-6-team_bds_p6_default  rack1       Up     Normal  87.75 KiB       100.00%             -9022953963321962585                        
p6-db-2.project-6-team_bds_p6_default  rack1       Up     Normal  87.75 KiB       100.00%             -8607316303750322705                        
p6-db-3.project-6-team_bds_p6_default  rack1       Up     Normal  87.74 KiB       100.00%             -8385076462431130982                        
p6-db-1             rack1       Up     Normal  87.75 KiB       100.00%             -7957379375437967804                        
p6-db-3.project-6-team_bds_p6_default  rack1       Up     Normal  87.74 KiB       100.00%             -7417712219352595483                        
p6-

In [27]:
x = subprocess.check_output(
    "nodetool ring -- weather",
    stderr=subprocess.STDOUT,
    shell=True)

In [28]:
x_str = x.decode('utf-8')

In [29]:
#q4
t = x_str.split("\n")
t = list(filter(lambda tx: 'Normal' in tx, t))
nodetool = []
for tx in t:
    values = list(map(lambda x: x.strip(), tx.split(' ')))
    nodetool.append([x for x in values if x != ''])
    # print(values)
    # print('-------------------')
tokens = []
for val in nodetool:
    tokens.append(int(val[-1]))
idx = 0
while(idx<len(tokens) and y>=tokens[idx]):
    idx += 1
tokens[idx-len(tokens)]

-8607316303750322705

In [31]:
tokens

[-9022953963321962585,
 -8607316303750322705,
 -8385076462431130982,
 -7957379375437967804,
 -7417712219352595483,
 -7395635957078341558,
 -6998313104729628767,
 -6338356079740924506,
 -5827707666838696845,
 -5739892388784185679,
 -5360210627224810431,
 -4705591742108781313,
 -4279305240827651923,
 -4208684371141653547,
 -3581576326401223302,
 -3230538918625539472,
 -2929103414252974039,
 -2485534426166285013,
 -1871726897324015822,
 -1451904617801952343,
 -1394237315474072753,
 -549860667982910997,
 -355862717567014054,
 249968002678219285,
 735434393125198205,
 890407561919672241,
 1508388593821872318,
 1579811040616359962,
 1961908178259705054,
 2572729583594935715,
 2979759583002132069,
 3020079270518943200,
 3797431515614482864,
 4091579886858976013,
 4392733884574882114,
 4873187396840062403,
 5109431291601403028,
 5609103089550956044,
 5927103224213753823,
 6269798214087868143,
 6688475896998557702,
 7002859105439333362,
 7438753137266882522,
 7738774798150227003,
 7931995902679

In [32]:
!python3 -m grpc_tools.protoc -I=. --python_out=. --grpc_python_out=. station.proto

In [33]:
import os
if not os.path.isdir('records.parquet'):
    !unzip records.zip

In [34]:
df = spark.read.parquet("records.parquet")

In [35]:
df.limit(10).toPandas()

,station,date,element,value
0,USW00014898,20220101,TMAX,-32.0
1,USW00014898,20220102,TMAX,-77.0
2,USW00014898,20220103,TMAX,-60.0
3,USW00014898,20220104,TMAX,0.0
4,USW00014898,20220105,TMAX,-16.0
5,USW00014898,20220106,TMAX,-71.0
6,USW00014898,20220107,TMAX,-71.0
7,USW00014898,20220108,TMAX,-32.0
8,USW00014898,20220109,TMAX,-27.0
9,USW00014898,20220110,TMAX,-149.0


In [36]:
df.filter(df.element.isin('TMAX', 'TMIN')).show()

+-----------+--------+-------+------+
|    station|    date|element| value|
+-----------+--------+-------+------+
|USW00014898|20220101|   TMAX| -32.0|
|USW00014898|20220102|   TMAX| -77.0|
|USW00014898|20220103|   TMAX| -60.0|
|USW00014898|20220104|   TMAX|   0.0|
|USW00014898|20220105|   TMAX| -16.0|
|USW00014898|20220106|   TMAX| -71.0|
|USW00014898|20220107|   TMAX| -71.0|
|USW00014898|20220108|   TMAX| -32.0|
|USW00014898|20220109|   TMAX| -27.0|
|USW00014898|20220110|   TMAX|-149.0|
|USW00014898|20220111|   TMAX| -16.0|
|USW00014898|20220112|   TMAX|   6.0|
|USW00014898|20220113|   TMAX|  11.0|
|USW00014898|20220114|   TMAX| -77.0|
|USW00014898|20220115|   TMAX| -99.0|
|USW00014898|20220116|   TMAX| -60.0|
|USW00014898|20220117|   TMAX| -21.0|
|USW00014898|20220118|   TMAX|  28.0|
|USW00014898|20220119|   TMAX|  28.0|
|USW00014898|20220120|   TMAX|-121.0|
+-----------+--------+-------+------+
only showing top 20 rows



In [37]:
df2 = df.groupBy(["station","date"]).pivot("element", ["TMIN", "TMAX"]).min("value")

In [38]:
df2.show()

+-----------+--------+------+------+
|    station|    date|  TMIN|  TMAX|
+-----------+--------+------+------+
|USW00014898|20220107|-166.0| -71.0|
|USW00014839|20220924| 117.0| 194.0|
|USW00014839|20220523|  83.0| 150.0|
|USW00014839|20221019|  11.0|  83.0|
|USW00014839|20220529| 139.0| 261.0|
|USR0000WDDG|20221130|-106.0| -39.0|
|USR0000WDDG|20220119|-178.0| -56.0|
|USW00014837|20220222| -88.0| -38.0|
|USR0000WDDG|20220202|-150.0|-106.0|
|USW00014839|20220427|   0.0|  39.0|
|USW00014839|20220708| 189.0| 222.0|
|USW00014839|20220917| 200.0| 294.0|
|USW00014837|20220624| 200.0| 322.0|
|USW00014898|20220129|-116.0| -60.0|
|USW00014839|20220715| 156.0| 233.0|
|USR0000WDDG|20220224|-128.0| -61.0|
|USR0000WDDG|20220130|-117.0| -33.0|
|USR0000WDDG|20220414| -17.0|  50.0|
|USW00014898|20220728| 156.0| 256.0|
|USW00014837|20220906| 117.0| 256.0|
+-----------+--------+------+------+
only showing top 20 rows



In [39]:
df2.count()

1460

In [42]:
#q5
import grpc
import station_pb2
import station_pb2_grpc

channel = grpc.insecure_channel("localhost:5440")
stub = station_pb2_grpc.StationStub(channel)

data = df2.collect()
for row in data:
    id = row['station']
    date = row['date'][:4]+'-'+row['date'][4:6]+'-'+row['date'][6:]
    tmin = int(row['TMIN'])
    tmax = int(row['TMAX'])
    res = stub.RecordTemps(station_pb2.RecordTempsRequest(station=id, date=date, tmin=tmin, tmax=tmax))
response = stub.StationMax(station_pb2.StationMaxRequest(station='USW00014837'))
response.tmax

356

In [43]:
stations_spark_df = spark.read.format("org.apache.spark.sql.cassandra") \
    .option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3") \
    .option("keyspace", "weather") \
    .option("table", "stations") \
    .load()

In [44]:
stations_spark_df.createOrReplaceTempView("stations")

In [45]:
#q6
spark.catalog.listTables()

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [46]:
#q7
temp_records = stations_spark_df.filter(col("record").isNotNull())
temp_diff = temp_records.withColumn("temp_diff", col("record.tmax") - col("record.tmin"))
avg_diff = temp_diff.groupBy("id") \
    .agg({"temp_diff": "avg"}) \
    .withColumnRenamed("avg(temp_diff)", "avg_diff")

result_dict = dict(avg_diff.orderBy("avg_diff").rdd.map(lambda x: (x["id"], x["avg_diff"])).collect())
result_dict

{'USW00014839': 89.6986301369863,
 'USR0000WDDG': 102.06849315068493,
 'USW00014898': 102.93698630136986,
 'USW00014837': 105.62739726027397}

23/11/22 23:39:12 WARN ChannelPool: [s0|p6-db-2/192.168.144.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=9e6802fd-3385-4d85-95b4-867765c178c4, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700696068564}): failed to send request (java.nio.channels.NotYetConnectedException))
23/11/22 23:39:19 WARN ChannelPool: [s0|p6-db-2/192.168.144.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=9e6802fd-3385-4d85-95b4-867765c178c4, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700696068564}): failed to send request (java.nio.channels.NotYetConnectedException))
23/11/22 23:39:28 WARN ChannelPool: [s0|p6

In [47]:
#q8
!nodetool status

Traceback (most recent call last):
  File "cassandra/cluster.py", line 3541, in cassandra.cluster.ControlConnection._reconnect_internal
  File "cassandra/cluster.py", line 3563, in cassandra.cluster.ControlConnection._try_connect
  File "cassandra/cluster.py", line 1630, in cassandra.cluster.Cluster.connection_factory
  File "cassandra/connection.py", line 850, in cassandra.connection.Connection.factory
  File "/usr/local/lib/python3.10/dist-packages/cassandra/io/asyncorereactor.py", line 347, in __init__
    self._connect_socket()
  File "cassandra/connection.py", line 917, in cassandra.connection.Connection._connect_socket
OSError: [Errno None] Tried connecting to [('192.168.144.2', 9042)]. Last error: timed out


Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address        Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.144.3  87.75 KiB  16      100.0%            ec7f5b01-8ee7-493e-bea3-ef75c949e4b8  rack1
UN  192.168.144.4  87.74 KiB  16      100.0%            14acab58-d159-4c7f-a753-2a65b482ca27  rack1
DN  192.168.144.2  87.75 KiB  16      100.0%            5c25eae9-4e89-4bf0-a856-db1b75d86ba8  rack1



In [48]:
#q9
request = station_pb2.StationMaxRequest(station='USW00014837')
response = stub.StationMax(request)
# print(type(response))
response.error

'need 3 replicas, but only have 2'

23/11/22 23:39:41 WARN ChannelPool: [s0|p6-db-2/192.168.144.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=9e6802fd-3385-4d85-95b4-867765c178c4, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700696068564}): failed to send request (java.nio.channels.NotYetConnectedException))


In [49]:
response = stub.StationMax(station_pb2.StationMaxRequest(station='USW00014837'))

In [50]:
#q10
request = station_pb2.RecordTempsRequest(station='USW00014839', date='2023-11-22', tmin=-100, tmax=100)
response = stub.RecordTemps(request)
# print(type(response))
response.error

''

23/11/22 23:39:58 WARN ChannelPool: [s0|p6-db-2/192.168.144.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=9e6802fd-3385-4d85-95b4-867765c178c4, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700696068564}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
23/11/22 23:40:34 WARN ChannelPool: [s0|p6-db-2/192.168.144.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=9e6802fd-3385-4d85-95b4-867765c178c4, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700696068564}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.Stackles